In [1]:
!pip install transformers
!pip install accelerate
!pip install datasets
!pip install bitsandbytes
!pip install peft
!pip install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 6.5 MB/s eta 0:00:00


In [1]:
from huggingface_hub import notebook_login

notebook_login()

### Load Dataset

In [2]:
from datasets import load_dataset, concatenate_datasets

datasets_names = [
    "medalpaca/medical_meadow_mediqa",
    # "medalpaca/medical_meadow_mmmlu",
    # "medalpaca/medical_meadow_medical_flashcards",
    ##############################################"medalpaca/medical_meadow_wikidoc_patient_information",
    # "medalpaca/medical_meadow_wikidoc",
    # "epfl-llm/guidelines", # newly added from meditron open dataset
    # "medalpaca/medical_meadow_medqa",
    ###############################################"medalpaca/medical_meadow_health_advice",
    ###############################################"medalpaca/medical_meadow_cord19",
]

# Load and combine datasets
dataset = [load_dataset(name, split="train") for name in datasets_names]
dataset = concatenate_datasets(dataset)
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Dataset({
    features: ['input', 'instruction', 'output'],
    num_rows: 2208
})

In [3]:
# from datasets import load_dataset

# dataset = load_dataset("TokenBender/code_instructions_122k_alpaca_style", split="train")
# dataset

# test_size = 0.2

# dataset = dataset.train_test_split(test_size=test_size)
# dataset

In [4]:
test_size = 0.2

dataset = dataset.train_test_split(test_size=test_size)
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'instruction', 'output'],
        num_rows: 1766
    })
    test: Dataset({
        features: ['input', 'instruction', 'output'],
        num_rows: 442
    })
})

In [5]:
train_dataset = dataset["train"]
test_dataset = dataset["test"]

In [6]:
train_dataset.to_pandas().head(5)

,input,instruction,output
0,Dementia Overview Dementia isn't a specific d...,how do i prevent myself from having dementia,"Most types of dementia can't be cured, but the..."
1,Primary lymphoma of the brain Brain lymphoma ...,what is the treatment for lymphoma,Primary lymphoma of the brain is usually first...
2,Preeclampsia (Prevention): There is no known w...,prevention of preeclampsia I know that the jur...,There is no known way to prevent preeclampsia....
3,What to do for Radiation sickness?: Unfortunat...,how do we treat the burns caused by radiation?,Report any radiation exposure to emergency off...
4,Adrenoleukodystrophy X-linked Adrenoleukodyst...,adrenoleukodystrophy A child I have been asked...,Adrenoleukodystrophy is usually passed down fr...


### Load Tokenizer

In [7]:
model_id = "mistralai/Mistral-7B-Instruct-v0.1"


In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

### Load Model

In [9]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_quant_type="nf4",
                                bnb__4bit_compute_dtype=torch.bfloat16)

In [10]:
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"": 0})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
def get_answer(query: str, model, tokenizer) -> str:
    device = "cuda"

    prompt_template = """
    [INST]
    Below is an instruction that describes a task. Write a response that appropriately completes the request.
    {query}
    [/INST]
    """

    prompt = prompt_template.format(query=query)

    encodeds = tokenizer(prompt,
                         return_tensors="pt",
                         add_special_tokens=True)

    model_inputs = encodeds.to(device)


    generated_ids = model.generate(**model_inputs,
                                   max_new_tokens=1000,
                                   do_sample=True,
                                   pad_token_id=tokenizer.eos_token_id)

    decoded = tokenizer.batch_decode(generated_ids)
    return (decoded[0])

In [12]:
query="EPIDEMIOLOGY & DEMOGRAPHICS of abdominal Abdominal Aortic Aneurysm ?"

result = get_answer(query=query,
                    model=model,
                    tokenizer=tokenizer)

print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


<s> 
    [INST]
    Below is an instruction that describes a task. Write a response that appropriately completes the request.
    EPIDEMIOLOGY & DEMOGRAPHICS of abdominal Abdominal Aortic Aneurysm ?
    [/INST]
    </s> Ultimately, the epidemiology and demographics of abdominal aortic aneurysms (AAA) suggest that the risk of developing an aneurysm increases as age advances, and is higher in men than women. AAA is most commonly seen in individuals over the age of 65, and men between the ages of 65 and 74 have a higher risk of developing an aneurysm than any other age group. Other risk factors include smoking, a family history of aneurysms, hypertension, and certain genetic factors. In terms of demographics, AAA is more common in individuals of European descent, and the incidence of the condition is highest in African Americans. Additionally, certain socioeconomic factors, such as access to healthcare and education, have been found to influence the incidence of AAA.</s>


### Generate Prompt Format

In [13]:
def format_prompt(data_point):
    """Gen. input text based on a prompt, task instruction, (context info.), and answer

    :param data_point: dict: Data point
    :return: dict: tokenzed prompt
    """

    prefix_text = """
    Answer this medical guidelines in english.
    if no answer: "i do not know."
    \n\n
    """

    # # Samples with additional context into.
    # if data_point["input"]:
    #     text = f"""[INST]{prefix_text} Context: {data_point["instruction"]} Question: {data_point["input"]} [/INST]{data_point["output"]}"""
    # # Without
    # else:
    #     text = f"""[INST]{prefix_text} Context: {data_point["instruction"]} [/INST]{data_point["output"]}"""

    # Samples with additional context into.
    if data_point["instruction"]:
        text = f"""[INST]{prefix_text} Context: {data_point["input"]}\nQuestion: {data_point["instruction"]} [/INST]\n{data_point["output"]}"""
    # Without
    else:
        text = f"""[INST]{prefix_text} Context: {data_point["input"]} [/INST]\n{data_point["output"]}"""

    return text


In [14]:
def generate_prompt(dataset, tokenizer):
    # add the "prompt" column in the dataset
    text_column = [format_prompt(data_point) for data_point in dataset]
    dataset = dataset.add_column("prompt", text_column)

    dataset = dataset.shuffle(seed=2)
    dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

    return dataset

In [15]:
train_dataset = generate_prompt(train_dataset, tokenizer)
test_dataset = generate_prompt(test_dataset, tokenizer)

Flattening the indices:   0%|          | 0/1766 [00:00<?, ? examples/s]

Map:   0%|          | 0/1766 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/442 [00:00<?, ? examples/s]

Map:   0%|          | 0/442 [00:00<?, ? examples/s]

In [16]:
print(test_dataset)

Dataset({
    features: ['input', 'instruction', 'output', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 442
})


In [19]:
print(test_dataset['prompt'][2])

[INST]
    Answer this medical guidelines in english.
    if no answer: "i do not know." 
    


     Context: Generalized anxiety disorder (Diagnosis): To help diagnose generalized anxiety disorder, your doctor or mental health professional may: - Do a physical exam to look for signs that your anxiety might be linked to medications or an underlying medical condition - Order blood or urine tests or other tests, if a medical condition is suspected - Ask detailed questions about your symptoms and medical history - Use psychological questionnaires to help determine a diagnosis - Use the criteria listed in the Diagnostic and Statistical Manual of Mental Disorders (DSM-5), published by the American Psychiatric Association
Question: how is generalized anxiety disorder diagnosed [/INST]
To help diagnose generalized anxiety disorder, your doctor or mental health professional will be able to take a few approaches: They will look for signs that link your anxiety to medications or other medical c

### Apply Lora

In [20]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)


In [21]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )

In [22]:
import bitsandbytes as bnb

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
        if "lm_head" in lora_module_names: # Needed for 16bit
            lora_module_names.remove("lm_head")

    return list(lora_module_names)


In [23]:
modules = find_all_linear_names(model)
print(modules)

['q_proj', 'gate_proj', 'up_proj', 'down_proj', 'k_proj', 'o_proj', 'v_proj']


In [24]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(r=8,
                         lora_alpha=32,
                         target_modules=modules,
                         lora_dropout=0.05,
                         bias="none",
                         task_type="CAUSAL_LM")

model = get_peft_model(model, lora_config)

In [25]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 20971520 | total: 7262703616 | Percentage: 0.2888%


### Define Training Arguments

In [26]:
from transformers import TrainingArguments

output_dir = "FineTune_Mixtral-7B-Instruct-v0.1"

training_arguments = TrainingArguments(output_dir=output_dir,
                                       warmup_steps=1,
                                       per_device_train_batch_size=20,
                                       per_device_eval_batch_size=20,
                                       gradient_accumulation_steps=4,
                                       gradient_checkpointing=True,
                                       num_train_epochs=1,            # 10 takes too long 71 hours
                                       learning_rate=2.5e-4,          # Want a small lr for finetuning
                                       lr_scheduler_type="cosine",
                                       # bf16=True,
                                       optim="paged_adamw_8bit",
                                       logging_steps=100,             # When to start reporting loss
                                       logging_dir="./logs",          # Directory for storing logs
                                       save_strategy="epoch",         # Save the model checkpoint every logging step
                                       save_steps=100,                # Save checkpoints every 50 steps
                                       eval_steps=100,                # Evaluate and save checkpoints every 50 steps
                                       evaluation_strategy="steps",   # Evaluate the model every logging step
                                       # do_eval=True,                # Perform evaluation at the end of training
                                       # max_steps=500,
                                       push_to_hub=True)


### Define Data Collator

In [27]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

### Define Trainer

In [28]:
# trainer = Trainer(model=model,
#                   train_dataset=train_dataset,
#                   eval_dataset=test_dataset,
#                   args=training_arguments,
#                   data_collator=data_collator)

In [29]:
from trl import SFTTrainer

trainer = SFTTrainer(model=model,
                     train_dataset=train_dataset,
                     eval_dataset=test_dataset,
                     dataset_text_field="prompt",
                     peft_config=lora_config,
                     args=training_arguments,
                     data_collator=data_collator)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:222: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1766 [00:00<?, ? examples/s]

Map:   0%|          | 0/442 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:284: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [30]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.50 GiB. GPU 0 has a total capacty of 14.75 GiB of which 411.06 MiB is free. Process 61381 has 14.34 GiB memory in use. Of the allocated memory 10.62 GiB is allocated by PyTorch, and 3.59 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
new_model = "Mixtral-7B-Instruct-v0.1-Medical-Finetune"

In [ ]:
trainer.model.save_pretrained(new_model)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(model_id,
                                                  low_cpu_mem_usage=True,
                                                  return_dict=True,
                                                  torch_dtype=torch.float16,
                                                  device_map={"":0})

merged_model = PeftModel.from_pretrained(base_model, new_model)
merged_model = merged_model.merge_and_upload()

# Save the merged model
merged_model.save_pretrained("merged_model", safe_serialiaztion=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# Push the model and tokenizer to the Hugging Face Model Hub
merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

### Evaluation

In [ ]:
def get_completion_merged(query: str, model, tokenizer) -> str:
  device = "cuda"

  prompt_template = """
  [INST]
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  {query}
  [/INST]
  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)

  generated_ids = merged_model.generate(**model_inputs,
                                        max_new_tokens=1000,
                                        do_sample=True,
                                        pad_token_id=tokenizer.eos_token_id)

  decoded = tokenizer.batch_decode(generated_ids)
  return (decoded[0])



In [ ]:
query="EPIDEMIOLOGY & DEMOGRAPHICS of abdominal Abdominal Aortic Aneurysm ?"

result = get_answer(query=query,
                    model=model,
                    tokenizer=tokenizer)
print(result)